In [419]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import re
import math
from selenium.webdriver.common.action_chains import ActionChains
import time
import json
import os, sys
import random
from datetime import date, timedelta
 


In [420]:
options = webdriver.ChromeOptions()
# options.headless = True
options.add_argument("window-size=1920x1080")

if getattr(sys, 'frozen', False):
    chromedriver_path = os.path.join(sys._MEIPASS, "chromedriver.exe")
    driver = webdriver.Chrome(chromedriver_path,options=options)
else:
    driver = webdriver.Chrome(options=options)


In [421]:
today = date.today()
yesterday = today - timedelta(days = 1)

In [422]:
### 블로그로 이동

driver.get('https://blog.naver.com/PostList.nhn?blogId=wishia&categoryNo=0&from=postList')

time.sleep(2)

In [423]:
### 리트스 박스 열기

driver.find_element_by_xpath('//*[@id="listCountToggle"]/i[2]').click()

time.sleep(0.5)

In [424]:
### 30개씩 보기 고르기

driver.find_element_by_xpath('//*[@id="changeListCount"]/a[5]').click()

In [425]:
### 전체 게시글의 수로 페이지 수 가져오기

article_num_base=driver.find_element_by_xpath('//*[@id="category-name"]/div/table[2]/tbody/tr/td[2]/div/h4').text

article_num = re.sub('[^0-9]','', article_num_base)
page = math.ceil(int(article_num)/30)

In [426]:
### 페이지 넘어가는 버튼

num_1=[]
num_2=[]
for i in range(1,11):
    num_1.append(i)

for i in range(2,12):
    num_2.append(i)

num_2 = num_2*100

num = num_1+num_2

In [428]:
title_all = []
date_all = []
url_all = []

page_int = 0

for i in num:
    
    page_int = page_int+1
    
    ### 제목 가져오기
    title = driver.find_elements_by_xpath('//*[@id="listTopForm"]/table/tbody/tr[*]/td[1]/div/span/a')
    for k in title:
        title_all.append(k.text)
    
    
    url = driver.find_elements_by_xpath('//*[@id="listTopForm"]/table/tbody/tr[*]/td[1]/div/span/a')

    ###URL 가져오기
    for j in url:
        url_all.append(j.get_attribute("href"))
        
    ### 날짜 가져오기
    date=driver.find_elements_by_xpath('//*[@id="listTopForm"]/table/tbody/tr[*]/td[2]/div/span')
    
    for q in date:
        date_base = q.text
        
        if len(re.findall('[가-힣]',date_base))>0:
            if time.localtime()[3]-int(re.sub('[^0-9]','',date_base))<0:
                article_date = yesterday.strftime('%Y.%m.%d.')
            else :
                article_date = today.strftime('%Y.%m.%d.')
        else:        
            if re.sub(' ','',date_base)[8]=='.':
                article_date = re.sub(' ','',date_base)[:8]
            elif re.sub(' ','',date_base)[9]=='.':
                article_date = re.sub(' ','',date_base)[:9]
            else:
                article_date = re.sub(' ','',date_base)[:10]

        date_all.append(article_date)


    
    if page_int == page:
        break
    
    driver.find_element_by_xpath('//*[@id="toplistWrapper"]/div[2]/div/a[%d]'%i).click()
    time.sleep(1)
    
driver.quit()


In [429]:
date_all

['2021.4.26',
 '2021.4.16',
 '2021.4.15',
 '2021.4.13',
 '2021.4.12',
 '2021.3.31',
 '2021.3.31',
 '2021.3.30',
 '2021.3.30',
 '2021.3.28',
 '2021.3.23',
 '2021.3.18',
 '2021.3.15',
 '2021.3.13',
 '2021.2.28',
 '2021.2.27',
 '2021.2.26',
 '2021.2.22',
 '2021.2.21',
 '2021.2.18',
 '2021.2.17',
 '2021.2.16',
 '2021.2.16',
 '2021.2.15',
 '2021.2.14',
 '2021.2.14',
 '2021.2.12',
 '2021.2.11',
 '2021.2.10',
 '2021.2.9',
 '2021.2.8',
 '2021.2.8',
 '2021.2.7',
 '2021.2.7',
 '2021.2.5',
 '2021.2.4',
 '2021.2.4',
 '2021.2.3',
 '2021.2.2',
 '2021.2.1',
 '2021.1.31',
 '2021.1.30',
 '2021.1.29',
 '2021.1.29',
 '2021.1.27',
 '2021.1.26',
 '2021.1.25',
 '2021.1.25',
 '2021.1.23',
 '2021.1.22',
 '2021.1.21',
 '2021.1.18',
 '2021.1.17',
 '2021.1.16',
 '2021.1.15',
 '2021.1.14',
 '2021.1.11',
 '2021.1.9',
 '2021.1.8',
 '2021.1.6',
 '2020.12.31',
 '2020.12.30',
 '2020.12.23',
 '2020.12.22',
 '2020.12.20',
 '2020.12.16',
 '2020.12.10',
 '2020.12.9',
 '2020.12.6',
 '2020.12.3',
 '2020.12.2',
 '2020.12.1',

In [430]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
}

In [440]:
page = 0
content_all=[]
check_list=[]
for url in url_all:

    page= page+1
    
    print('진행 경과 : '+str(page)+'/'+str(len(url_all)))

    res = requests.get(url, headers=header)

    if res.status_code==200:
        check_list.append('정상')
    else:
        check_list.append('에러')
        continue

    ### 2페이지 이상부터 페이지 정보 안가져와졌으면 다시 가져오기

    if page>1:
        if soup==BeautifulSoup(res.content, 'html.parser'):
            while True:
                res = requests.get(url_date, headers=header)
                if soup!=BeautifulSoup(res.content, 'html.parser'):
                    break
                    
    soup=BeautifulSoup(res.content, 'html.parser')



    ### 내용 가져오기
    
    try:
        content=soup.find('div',{'class':'se-main-container'}).text
    except:
        try:
            content=soup.find('div',{'class':'se_component_wrap sect_dsc __se_component_area'}).text
        except:
            content=soup.find('div',{'id':'postViewArea'}).text

    
    content_all.append(content)


df = {"날짜":date_all,"제목":title_all,'본문':content_all ,"URL" : url_all}
df = pd.DataFrame(df)

nows = time.localtime()
times = str(nows.tm_year) + '-' + str(nows.tm_mon) + '-' + str(nows.tm_mday)
name = times + '블로그 수집' +'.xlsx'

writer = pd.ExcelWriter(name, engine='xlsxwriter', options={'strings_to_urls': True})
df.to_excel(writer)
writer.close()

진행 경과 : 1/796
진행 경과 : 2/796
진행 경과 : 3/796
진행 경과 : 4/796
진행 경과 : 5/796
진행 경과 : 6/796
진행 경과 : 7/796
진행 경과 : 8/796
진행 경과 : 9/796
진행 경과 : 10/796
진행 경과 : 11/796
진행 경과 : 12/796
진행 경과 : 13/796
진행 경과 : 14/796
진행 경과 : 15/796
진행 경과 : 16/796
진행 경과 : 17/796
진행 경과 : 18/796
진행 경과 : 19/796
진행 경과 : 20/796
진행 경과 : 21/796
진행 경과 : 22/796
진행 경과 : 23/796
진행 경과 : 24/796
진행 경과 : 25/796
진행 경과 : 26/796
진행 경과 : 27/796
진행 경과 : 28/796
진행 경과 : 29/796
진행 경과 : 30/796
진행 경과 : 31/796
진행 경과 : 32/796
진행 경과 : 33/796
진행 경과 : 34/796
진행 경과 : 35/796
진행 경과 : 36/796
진행 경과 : 37/796
진행 경과 : 38/796
진행 경과 : 39/796
진행 경과 : 40/796
진행 경과 : 41/796
진행 경과 : 42/796
진행 경과 : 43/796
진행 경과 : 44/796
진행 경과 : 45/796
진행 경과 : 46/796
진행 경과 : 47/796
진행 경과 : 48/796
진행 경과 : 49/796
진행 경과 : 50/796
진행 경과 : 51/796
진행 경과 : 52/796
진행 경과 : 53/796
진행 경과 : 54/796
진행 경과 : 55/796
진행 경과 : 56/796
진행 경과 : 57/796
진행 경과 : 58/796
진행 경과 : 59/796
진행 경과 : 60/796
진행 경과 : 61/796
진행 경과 : 62/796
진행 경과 : 63/796
진행 경과 : 64/796
진행 경과 : 65/796
진행 경과 : 66/796
진행 경과 : 67/796
진행 경

진행 경과 : 520/796
진행 경과 : 521/796
진행 경과 : 522/796
진행 경과 : 523/796
진행 경과 : 524/796
진행 경과 : 525/796
진행 경과 : 526/796
진행 경과 : 527/796
진행 경과 : 528/796
진행 경과 : 529/796
진행 경과 : 530/796
진행 경과 : 531/796
진행 경과 : 532/796
진행 경과 : 533/796
진행 경과 : 534/796
진행 경과 : 535/796
진행 경과 : 536/796
진행 경과 : 537/796
진행 경과 : 538/796
진행 경과 : 539/796
진행 경과 : 540/796
진행 경과 : 541/796
진행 경과 : 542/796
진행 경과 : 543/796
진행 경과 : 544/796
진행 경과 : 545/796
진행 경과 : 546/796
진행 경과 : 547/796
진행 경과 : 548/796
진행 경과 : 549/796
진행 경과 : 550/796
진행 경과 : 551/796
진행 경과 : 552/796
진행 경과 : 553/796
진행 경과 : 554/796
진행 경과 : 555/796
진행 경과 : 556/796
진행 경과 : 557/796
진행 경과 : 558/796
진행 경과 : 559/796
진행 경과 : 560/796
진행 경과 : 561/796
진행 경과 : 562/796
진행 경과 : 563/796
진행 경과 : 564/796
진행 경과 : 565/796
진행 경과 : 566/796
진행 경과 : 567/796
진행 경과 : 568/796
진행 경과 : 569/796
진행 경과 : 570/796
진행 경과 : 571/796
진행 경과 : 572/796
진행 경과 : 573/796
진행 경과 : 574/796
진행 경과 : 575/796
진행 경과 : 576/796
진행 경과 : 577/796
진행 경과 : 578/796
진행 경과 : 579/796
진행 경과 : 580/796
진행 경과 : 581/796
진행 경과 : 